# Sound propagation model data processing

This notebook brings in the sound propagation modeling results and links them to the occurrence records.

Sound propagation modeling data are available on Google Cloud at https://console.cloud.google.com/storage/browser/noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models;tab=objects?prefix=&forceOnObjectsSortingFiltering=false

We can use the [`gutil cp`](https://cloud.google.com/storage/docs/gsutil/commands/cp) command to recursively download files from Google Cloud to a local directory.

```
gutil cp -r gs://noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models/ci01/sanctsound_ci01_propmodeling/data/
```

But we only want the netCDF data and we only want the `sound_propagation` value.

```
ds = xr.open_dataset('SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Apr_radarformat_highres.nc')
ds.variables['listening_range']

Out[6]:
<xarray.Variable (month: 1)>
array([2815])
Attributes:
    long_name:    distance_from_hydrophone_to_zero_SNR
    Description:  The median distance from the hydrophone to a zero signal-to...
    units:        m
```



In [1]:
import pandas as pd
import xarray as xr

# Function to download public files.

From https://cloud.google.com/storage/docs/access-public-data#storage-download-public-object-python

```
conda install google-cloud-storage
```

https://console.cloud.google.com/storage/browser/_details/noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models/ci01/sanctsound_ci01_propmodeling/data/SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Apr_radarformat_highres.nc;tab=live_object

In [2]:
# Download the readme for the noaa-passive-bioacoustic bucket

from google.cloud import storage

storage_client = storage.Client.create_anonymous_client()

bucket_name = 'noaa-passive-bioacoustic'
delimiter='/'
bucket=storage_client.get_bucket(bucket_name)
blobs=bucket.list_blobs(delimiter=delimiter) #List all objects that satisfy the filter.

for blob in blobs:
    print(blob.name)
    blob.download_to_filename(blob.name)

README.pdf


In [3]:
from IPython.display import IFrame

IFrame(blob.name, width=900, height=1200)

Install `gsutil` from https://cloud.google.com/storage/docs/gsutil_install

and recusively download netCDF files from `noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models`

gsutil uri for one of the datasets is:
```
gs://noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models/ci01/sanctsound_ci01_propmodeling/data/SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Apr_radarformat_highres.nc
```

Data citation:
```
NOAA National Centers for Environmental Information. 2017. Passive Acoustic Data Collection. NOAA National Centers for Environmental Information.
https://doi.org/10.25921/PF0H-SQ72. access date
```

In [36]:
# for one station
#!gcloud storage ls gs://noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models/sb03/**/*.nc
    
# for all stations

temp = !gcloud storage ls gs://noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models/**/*.nc

print(f'Found {len(temp)} files:')    

for file in temp:
    print(file.split('/')[-1])

Found 924 files:
SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Apr_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Jan_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Jul_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Oct_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ05000Hz_Apr_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ05000Hz_Jan_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ05000Hz_Jul_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ05000Hz_Oct_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0010m_SL185dB_FQ00125Hz_Apr_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0010m_SL185dB_FQ00125Hz_Jan_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0010m_SL185dB_FQ00125Hz_Jul_radarformat_highres.nc
SanctSound_CI01_propmodeling_SD0010m_SL185dB_FQ00125Hz_Oct_radarformat_highres.nc

In [5]:
## BE CAREFUL WITH THIS >900 FILES!
#
# !gsutil cp gs://noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models/**/*.nc data\sound_propagation\

In [6]:
# Might be able to use xarray to grab data from gc:

# url = 'gs://noaa-passive-bioacoustic/sanctsound/products/sound_propagation_models/ci01/sanctsound_ci01_propmodeling/data/SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Apr_radarformat_highres.nc'

# xr.open_dataset(url,
#                 engine="netcdf4"
#                )

# Match occurrence data with Sound propagation model

Now we can bring in the occurrence data and determine how to link the two resources together.

Reminder, occurrence data was collected from https://coastwatch.pfeg.noaa.gov/erddap/search/index.html?page=1&itemsPerPage=1000&searchFor=noaaSanctSound

In [7]:
df_occur = pd.read_csv('data/occurrence.zip', compression='zip')

df_occur.head()

,WKT,decimalLatitude,decimalLongitude,vernacularName,scientificName,scientificNameID,taxonRank,kingdom,eventDate,occurrenceID
0,POINT (31.396417 -80.8904),31.396417,-80.8904,dolphin,Cetacea,urn:lsid:marinespecies.org:taxname:2688,Infraorder,Animalia,2018-12-15T04:00:00.000000Z,noaaSanctSound_GR01_01_dolphins_1h_2018-12-15T...
1,POINT (31.396417 -80.8904),31.396417,-80.8904,dolphin,Cetacea,urn:lsid:marinespecies.org:taxname:2688,Infraorder,Animalia,2018-12-15T05:00:00.000000Z,noaaSanctSound_GR01_01_dolphins_1h_2018-12-15T...
2,POINT (31.396417 -80.8904),31.396417,-80.8904,dolphin,Cetacea,urn:lsid:marinespecies.org:taxname:2688,Infraorder,Animalia,2018-12-15T06:00:00.000000Z,noaaSanctSound_GR01_01_dolphins_1h_2018-12-15T...
3,POINT (31.396417 -80.8904),31.396417,-80.8904,dolphin,Cetacea,urn:lsid:marinespecies.org:taxname:2688,Infraorder,Animalia,2018-12-15T07:00:00.000000Z,noaaSanctSound_GR01_01_dolphins_1h_2018-12-15T...
4,POINT (31.396417 -80.8904),31.396417,-80.8904,dolphin,Cetacea,urn:lsid:marinespecies.org:taxname:2688,Infraorder,Animalia,2018-12-15T18:00:00.000000Z,noaaSanctSound_GR01_01_dolphins_1h_2018-12-15T...


Let's collect unique station and locality identifiers to match to propagation results.

In [8]:
ident = df_occur['occurrenceID'].str.split('_',expand=True).rename(columns={1:'station',2:'local'})

ident['concat'] = ident['station']+'_'+ident['local']

ident['concat'].unique()

array(['GR01_01', 'GR01_02', 'GR01_03', 'GR01_04', 'GR01_05', 'GR02_02',
       'GR02_03', 'GR02_05', 'GR03_02', 'GR03_03', 'GR03_04', 'GR03_05',
       'CI01_01', 'CI01_02', 'CI01_05', 'CI01_06', 'CI02_01', 'CI02_02',
       'CI02_05', 'CI02_06', 'CI02_07', 'CI03_04', 'CI03_05', 'CI03_06',
       'CI03_07', 'CI04_01', 'CI04_02', 'CI04_03', 'CI04_05', 'CI04_06',
       'CI04_07', 'CI04_08', 'CI05_01', 'CI05_02', 'CI05_03', 'CI05_06',
       'CI05_08', 'MB01_01', 'MB01_02', 'MB01_03', 'MB01_04', 'MB01_05',
       'MB01_06', 'MB01_07', 'MB01_08', 'MB01_09', 'MB02_01', 'MB02_03',
       'MB02_04', 'MB02_05', 'MB02_07', 'MB03_01', 'MB03_02', 'MB03_03',
       'MB03_04', 'OC02_02', 'OC03_02', 'OC03_03', 'OC04_04', 'SB02_06',
       'SB03_06', 'CI01_03', 'CI03_02', 'CI03_03', 'MB02_02', 'CI01_04',
       'CI02_04', 'CI04_04', 'CI05_04', 'CI05_05', 'FK01_01', 'FK01_04',
       'FK01_06', 'FK02_01', 'FK02_03', 'FK02_04', 'FK02_05', 'FK02_06',
       'FK03_01', 'FK03_03', 'FK03_04', 'FK03_05', 

# Investigate downloaded sound propagation files

We only downloaded a subset of the propagation model data for testing. There are **924** files, so we should explore how we might be able to do this work without downloading all the data.

What does our sound propagation model output look like? Let's look at the first file that we downloaded.

In [37]:
import os

directory = 'data/sound_propagation/'

fname = os.listdir(directory)[0]

print(fname,'\n')

ds = xr.open_dataset(directory+fname, engine='netcdf4')

ds.info()

SanctSound_CI01_propmodeling_SD0001m_SL165dB_FQ01000Hz_Apr_radarformat_highres.nc 

xarray.Dataset {
dimensions:
	month = 1 ;
	depth = 1 ;
	bearing = 361 ;
	range = 13853 ;

variables:
	float64 month(month) ;
		month:long_name = month_of_climatological_sound_speed_profiles ;
		month:description = month # for GDEM sound speed profiles: 1-Jan, 2-Feb etc ;
		month:units = 1 ;
	float64 depth(depth) ;
		depth:long_name = sound_source_depth ;
		depth:description = Depth of the sound source ;
		depth:units = m ;
	float64 bearing(bearing) ;
		bearing:axis = Y ;
		bearing:long_name = true_north_bearing_from_hydrophone ;
		bearing:units = degrees_true ;
	float64 range(range) ;
		range:axis = X ;
		range:long_name = range_away_from_hydrophone ;
		range:units = km ;
	|S4 site() ;
		site:long_name = SanctSound_site_name ;
	float64 latitude(bearing, range) ;
		latitude:standard_name = latitude ;
		latitude:units = degrees_north ;
	float64 longitude(bearing, range) ;
		longitude:standard_name = longi

We know we want the data from the variable `listening_range`.

In [11]:
ds['listening_range']

<xarray.DataArray 'listening_range' (month: 1)>
[1 values with dtype=int32]
Coordinates:
  * month    (month) float64 4.0
Attributes:
    long_name:    distance_from_hydrophone_to_zero_SNR
    Description:  The median distance from the hydrophone to a zero signal-to...
    units:        m

Grab all the `listening_range` data (in meters) with filenames to see if we can match to the occurrence data

In [21]:

df_listening_range = pd.DataFrame()

fnames = os.listdir(directory)

for fname in fnames:
    
    with xr.open_dataset(directory+fname, engine='netcdf4') as ds:
    
        df_temp = ds[['listening_range','depth','site']].to_dataframe().reset_index()

        # add additional data
        df_temp['fname'] = fname
        df_temp['site'] = df_temp['site'].values[0].decode('utf-8')
        df_temp['freq_Hz'] = int(ds.SoundSourcefrequency.replace('Hz','').strip())
        df_temp.rename(columns={'listening_range':'listening_range_m'}, inplace=True)

        df_listening_range = pd.concat([df_listening_range, df_temp])
        
df_listening_range

,month,depth,listening_range_m,site,fname,freq_Hz
0,4.0,1.0,2815,CI01,SanctSound_CI01_propmodeling_SD0001m_SL165dB_F...,1000
0,1.0,1.0,4593,CI01,SanctSound_CI01_propmodeling_SD0001m_SL165dB_F...,1000
0,7.0,1.0,3908,CI01,SanctSound_CI01_propmodeling_SD0001m_SL165dB_F...,1000
0,10.0,1.0,4398,CI01,SanctSound_CI01_propmodeling_SD0001m_SL165dB_F...,1000
0,4.0,1.0,2398,CI01,SanctSound_CI01_propmodeling_SD0001m_SL165dB_F...,5000
...,...,...,...,...,...,...
0,4.0,15.0,26984,SB03,SanctSound_SB03_propmodeling_SD0015m_SL189dB_F...,20
0,1.0,15.0,26391,SB03,SanctSound_SB03_propmodeling_SD0015m_SL189dB_F...,20
0,7.0,15.0,99378,SB03,SanctSound_SB03_propmodeling_SD0015m_SL189dB_F...,20
0,10.0,15.0,26382,SB03,SanctSound_SB03_propmodeling_SD0015m_SL189dB_F...,20


Print out the dataframe to share via chat

In [13]:
# pd.set_option('display.max_colwidth', None)

# columns = [ 'site','month','freq_Hz', 'depth', 'listening_range_m']

# print(df_listening_range.sort_values(by=columns, ascending=True).to_csv(columns=columns,index=False))

Find occurrence records associated with the station `CI01`.

In [38]:
df_occur.loc[df_occur['occurrenceID'].str.contains('CI01')]

,WKT,decimalLatitude,decimalLongitude,vernacularName,scientificName,scientificNameID,taxonRank,kingdom,eventDate,occurrenceID
3447,POINT (34.0438 -120.0811),34.0438,-120.0811,blue whale,Balaenoptera musculus,urn:lsid:marinespecies.org:taxname:137090,Species,Animalia,2018-11-07T22:02:01.456000Z,noaaSanctSound_CI01_01_bluewhale_2018-11-07T22...
3448,POINT (34.0438 -120.0811),34.0438,-120.0811,blue whale,Balaenoptera musculus,urn:lsid:marinespecies.org:taxname:137090,Species,Animalia,2018-11-15T15:18:02.648000Z,noaaSanctSound_CI01_01_bluewhale_2018-11-15T15...
3449,POINT (34.0438 -120.0811),34.0438,-120.0811,blue whale,Balaenoptera musculus,urn:lsid:marinespecies.org:taxname:137090,Species,Animalia,2018-11-15T15:18:55.896000Z,noaaSanctSound_CI01_01_bluewhale_2018-11-15T15...
3450,POINT (34.0438 -120.0811),34.0438,-120.0811,blue whale,Balaenoptera musculus,urn:lsid:marinespecies.org:taxname:137090,Species,Animalia,2018-11-15T15:19:49.144000Z,noaaSanctSound_CI01_01_bluewhale_2018-11-15T15...
3451,POINT (34.0438 -120.0811),34.0438,-120.0811,blue whale,Balaenoptera musculus,urn:lsid:marinespecies.org:taxname:137090,Species,Animalia,2018-11-15T15:22:30.936000Z,noaaSanctSound_CI01_01_bluewhale_2018-11-15T15...
...,...,...,...,...,...,...,...,...,...,...
709891,POINT (34.0436 -120.0803),34.0436,-120.0803,plainfin midshipman,Porichthys notatus,urn:lsid:marinespecies.org:taxname:275658,Species,Animalia,2021-09-07T02:57:13.500000Z,noaaSanctSound_CI01_08_plainfinmidshipman_2021...
709892,POINT (34.0436 -120.0803),34.0436,-120.0803,plainfin midshipman,Porichthys notatus,urn:lsid:marinespecies.org:taxname:275658,Species,Animalia,2021-09-08T02:59:29.500000Z,noaaSanctSound_CI01_08_plainfinmidshipman_2021...
709893,POINT (34.0436 -120.0803),34.0436,-120.0803,plainfin midshipman,Porichthys notatus,urn:lsid:marinespecies.org:taxname:275658,Species,Animalia,2021-09-09T03:05:29.500000Z,noaaSanctSound_CI01_08_plainfinmidshipman_2021...
709894,POINT (34.0436 -120.0803),34.0436,-120.0803,plainfin midshipman,Porichthys notatus,urn:lsid:marinespecies.org:taxname:275658,Species,Animalia,2021-09-10T02:59:30.500000Z,noaaSanctSound_CI01_08_plainfinmidshipman_2021...


Now find the **listening ranges** for station `CI01` for the January climatology (month = 1.0).

In [23]:
df_listening_range.loc[(df_listening_range['site']=='CI01') & (df_listening_range['month']==1.0)].sort_values(by=['freq_Hz'])

,month,depth,listening_range_m,site,fname,freq_Hz
0,1.0,15.0,4861,CI01,SanctSound_CI01_propmodeling_SD0015m_SL189dB_F...,20
0,1.0,20.0,15001,CI01,SanctSound_CI01_propmodeling_SD0020m_SL192dB_F...,63
0,1.0,10.0,18270,CI01,SanctSound_CI01_propmodeling_SD0010m_SL185dB_F...,125
0,1.0,20.0,9492,CI01,SanctSound_CI01_propmodeling_SD0020m_SL170dB_F...,300
0,1.0,1.0,4593,CI01,SanctSound_CI01_propmodeling_SD0001m_SL165dB_F...,1000
0,1.0,1.0,3028,CI01,SanctSound_CI01_propmodeling_SD0001m_SL165dB_F...,5000
0,1.0,19.0,7325,CI01,SanctSound_CI01_propmodeling_SD0019m_SL176dB_F...,12000


Let's look at the SanctSound website and see how we might be able to link these together.

https://sanctsound.portal.axds.co/#sanctsound/sanctuary/channel-islands/site/CI01

Lets subset the occurrence data and set the index as `eventDate` so we can use pandas timeseries features to align the data.

In [24]:
df_subset = df_occur.loc[df_occur['occurrenceID'].str.contains('CI01'),['eventDate','occurrenceID']]

df_subset['eventDate'] = pd.to_datetime(df_subset['eventDate'])

df_subset = df_subset.set_index('eventDate',drop=True)

df_subset

,occurrenceID
eventDate,
2018-11-07 22:02:01.456000+00:00,noaaSanctSound_CI01_01_bluewhale_2018-11-07T22...
2018-11-15 15:18:02.648000+00:00,noaaSanctSound_CI01_01_bluewhale_2018-11-15T15...
2018-11-15 15:18:55.896000+00:00,noaaSanctSound_CI01_01_bluewhale_2018-11-15T15...
2018-11-15 15:19:49.144000+00:00,noaaSanctSound_CI01_01_bluewhale_2018-11-15T15...
2018-11-15 15:22:30.936000+00:00,noaaSanctSound_CI01_01_bluewhale_2018-11-15T15...
...,...
2021-09-07 02:57:13.500000+00:00,noaaSanctSound_CI01_08_plainfinmidshipman_2021...
2021-09-08 02:59:29.500000+00:00,noaaSanctSound_CI01_08_plainfinmidshipman_2021...
2021-09-09 03:05:29.500000+00:00,noaaSanctSound_CI01_08_plainfinmidshipman_2021...


Since propagation model data are separated into quarterly observations on months 1, 4, 7, and 10. We can use pandas to group by quarters starting in January.

In [25]:
df_group = df_subset.groupby(pd.Grouper(freq='Q-JAN'))

s = df_group.count()

s.index = s.index.to_period("M")

s

C:\Users\Mathew.Biddle\AppData\Local\Temp\2\ipykernel_19436\2003495430.py:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  s.index = s.index.to_period("M")


,occurrenceID
eventDate,
2019-01,1845
2019-04,660
2019-07,703
2019-10,322
2020-01,3
2020-04,72
2020-07,99
2020-10,229
2021-01,112


Let's pull out the month information as it's own column to match to the propagation model month.

In [26]:
s = s.reset_index()

s['month'] = s['eventDate'].astype(str).str.split('-', expand=True)[1]

s

,eventDate,occurrenceID,month
0,2019-01,1845,01
1,2019-04,660,04
2,2019-07,703,07
3,2019-10,322,10
4,2020-01,3,01
5,2020-04,72,04
6,2020-07,99,07
7,2020-10,229,10
8,2021-01,112,01
9,2021-04,55,04


Now iterate through each quarter and give back the listening ranges for each of the frequencies that match our site and quarter.

In [27]:
for index, row in s.iterrows():
    #print(row)
    
    site = 'CI01'
    month = int(row['month'])
    
    list_range = df_listening_range.loc[
                        (df_listening_range['site']==site) & (df_listening_range['month']==month),
                        ['listening_range_m']
            ]
    
    string = list_range['listening_range_m'].tolist()
    
    print(f'Listening range for {site} {month}:\t{string}')

Listening range for CI01 1:	[4593, 3028, 18270, 4861, 7325, 9492, 15001]
Listening range for CI01 4:	[2815, 2398, 18270, 4815, 5371, 8816, 15001]
Listening range for CI01 7:	[3908, 2343, 18270, 4824, 6889, 8862, 15001]
Listening range for CI01 10:	[4398, 2695, 18270, 4815, 7565, 8908, 15001]
Listening range for CI01 1:	[4593, 3028, 18270, 4861, 7325, 9492, 15001]
Listening range for CI01 4:	[2815, 2398, 18270, 4815, 5371, 8816, 15001]
Listening range for CI01 7:	[3908, 2343, 18270, 4824, 6889, 8862, 15001]
Listening range for CI01 10:	[4398, 2695, 18270, 4815, 7565, 8908, 15001]
Listening range for CI01 1:	[4593, 3028, 18270, 4861, 7325, 9492, 15001]
Listening range for CI01 4:	[2815, 2398, 18270, 4815, 5371, 8816, 15001]
Listening range for CI01 7:	[3908, 2343, 18270, 4824, 6889, 8862, 15001]
Listening range for CI01 10:	[4398, 2695, 18270, 4815, 7565, 8908, 15001]


In [ ]:
# TODO link those ranges back into df_group and back into each datasetID in df_occurrence